In [1]:
import pandas as pd
import matplotlib 
from matplotlib import pyplot as plt
import numpy as np
import csv
import glob

def Read_Data(WeekNumber, Year):
#	pass
      columns=["Mode", "Date", "DateTime", "CardID", "CardType", "VehicleID",
                   "ParentRoute", "RouteID", "StopID"] 

     # fname = "{0}/{1}/{2}/{3}/".format(path, scan_type, year, week)
      fname = glob.glob(Year + '/Week%s' %WeekNumber + "/*.txt")
      print(fname)
      dataset = pd.read_csv(fname[0], sep="|", names=columns)
   
      # Creating separate columns for Year, Month, Day
      dataset['Date']  = pd.to_datetime(dataset['Date'])
      dataset['Year']  = dataset['Date'].dt.year
      dataset['Month'] = dataset['Date'].dt.month
      dataset['Day']   = dataset['Date'].dt.day

      #Reading in stoplocation data:
      columns2 = ["StopID", "StopName", "Bla", "Bla", "Bla", "PostCode", "City","Bla", "MetroName", "lat", "long"]
      dataset2 = pd.read_csv('stop_locations.txt', sep="|", names = columns2)
      # creating a file with all data and their matched latitued and longitude cooridnates.
      newdata =  dataset2[['StopID', 'StopName','PostCode', 'lat', 'long']]
      
      All = pd.merge(dataset, newdata, how='right', on='StopID')
      return All


def Append_data(Year, FirstWeek, LastWeek):
    basic_table = Read_Data(FirstWeek, Year)
    for i in range(FirstWeek+1, LastWeek+1):
          Actual_Table    = Read_Data(i, Year)
          basic_table     = basic_table.append(Actual_Table) 
          print('appended', len(basic_table))
    return basic_table

dataset_2016 = Append_data("2016", 2, 26)
dataset_2018 = Append_data("2018", 2, 26)



['2016/Week2/file2.txt']


/anaconda3/lib/python3.6/site-packages/pandas/io/parsers.py:678: UserWarning: Duplicate names specified. This will raise an error in the future.
  return _read(filepath_or_buffer, kwds)


['2016/Week3/file3.txt']
appended 619537
['2016/Week4/file4.txt']
appended 968760
['2016/Week5/file5.txt']
appended 1291069
['2016/Week6/file6.txt']
appended 1685100
['2016/Week7/file7.txt']
appended 2088917
['2016/Week8/file8.txt']
appended 2512923
['2016/Week9/file9.txt']
appended 2937628
['2016/Week10/file10.txt']
appended 3394113
['2016/Week11/file11.txt']
appended 3852849
['2016/Week12/file12.txt']
appended 4264176
['2016/Week13/file13.txt']
appended 4660013
['2016/Week14/file14.txt']
appended 4993958
['2016/Week15/file15.txt']
appended 5408086
['2016/Week16/file16.txt']
appended 5852886
['2016/Week17/file17.txt']
appended 6298087
['2016/Week18/file18.txt']
appended 6694650
['2016/Week19/file19.txt']
appended 7132494
['2016/Week20/file20.txt']
appended 7561155
['2016/Week21/file21.txt']
appended 7997684
['2016/Week22/file22.txt']
appended 8416934
['2016/Week23/file23.txt']
appended 8815724
['2016/Week24/file24.txt']
appended 9210636
['2016/Week25/file25.txt']
appended 9568634
['20

In [43]:
def group(mydata):  
    #since groupby gets rid of columns, we are gonna keep the following ones:
    dataset_loc       = mydata[["StopID", "StopName", "PostCode", "lat", "long"]] 
    # groupig the dataset by StopID, and counting the number of touch-ons!
    dataset_grouped   = mydata.groupby("StopID").CardID.count().reset_index()
    #dataset_grouped   = mydata.groupby("StopID").CardID.nunique().reset_index()

    All               = pd.merge(dataset_grouped, dataset_loc, how ='left', on = 'StopID')
    All.drop_duplicates(subset = "StopID", keep='first', inplace=True)
    return All

All_2016_grouped = group(dataset_2016)
All_2018_grouped = group(dataset_2018)


In [44]:
import folium 

dataset_2016_2018 = pd.merge(All_2016_grouped , All_2018_grouped , on = "StopID" ) 
# the map will not render if there are too many points
# sort it and take the highest 300
dataset_2016_2018 = dataset_2016_2018.sort_values(by=["CardID_x"]) 
dataset_2016_2018 = dataset_2016_2018.tail(300)

Delta_R = dataset_2016_2018.CardID_y - dataset_2016_2018.CardID_x #)np.abs(

# creating growth and decline datasets
dataset_growth  = dataset_2016_2018[Delta_R > 0]
dataset_decline = dataset_2016_2018[Delta_R < 0]


In [45]:
## growth 

Delta_R_growth     = dataset_growth.CardID_y - dataset_growth.CardID_x
total_number       = Delta_R_growth.count()
print('total number growth', total_number)
Delta_R_growth_ratio = Delta_R_growth / np.sqrt(total_number) 

# adding the measured ratio to the growth dataset
dataset_growth["Delta_R_growth"] = Delta_R_growth_ratio


#creating the folium map
m2 = folium.Map(location=[-37.9,145], tiles="openstreetmap",  zoom_start=9.5) #,
 
# I can add marker one by one on the map
for i in range(0,len(dataset_growth)):
   folium.Circle(
      location=[dataset_growth.iloc[i]['lat_x'], dataset_growth.iloc[i]['long_x']],
      popup=dataset_growth.iloc[i]['StopName_x'], 
      radius=dataset_growth.iloc[i]['Delta_R_growth'],
      color='blue',
      fill=True,
      fill_color='blue'
   ).add_to(m2)
 

total number growth 191


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [47]:
## decline  

Delta_R_decline     = dataset_decline.CardID_x - dataset_decline.CardID_y  
total_number        =  Delta_R_decline.count()
print('total number decline', total_number)
Delta_R_decline_ratio = Delta_R_decline / np.sqrt(total_number) 

# adding the ratio to the decline dataset
dataset_decline["Delta_R_decline"] = Delta_R_decline_ratio 
 
# adding these points to the map
for i in range(0,len(dataset_decline)):
   folium.Circle(
      location=[dataset_decline.iloc[i]['lat_x'], dataset_decline.iloc[i]['long_x']],
      popup=dataset_decline.iloc[i]['StopName_x'],
      radius=dataset_decline.iloc[i]['Delta_R_decline'],
      color='red',
      fill=True,
      fill_color='red'
   ).add_to(m2)
 

# Save it as html
m2.save('mymap_2016_2018_touchOn_All.html')


total number decline 109


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
